##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Exemplos de migração: estimadores predefinidos

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/canned_estimators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/canned_estimators.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/canned_estimators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/canned_estimators.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

Estimadores predefinidos (ou pré-fabricados) são tradicionalmente usados ​​no TensorFlow 1 como soluções rápidas e fáceis de treinar modelos para uma variedade de casos de uso típicos. O TensorFlow 2 fornece substitutos aproximados diretos para vários deles através de modelos Keras. Para os estimadores predefinidos que não possuem substitutos integrados no TensorFlow 2, você ainda pode criar sua própria substituição com bastante facilidade.

Este guia mostrará alguns exemplos de equivalentes diretos e substituições personalizadas para demonstrar como os modelos derivados de `tf.estimator` do TensorFlow 1 podem ser migrados para o TensorFlow 2 com Keras.

Ou seja, este guia inclui exemplos para migração:

- De `tf.estimator`, `Classifier` ou `Regressor` `LinearEstimator` tf.estimator no TensorFlow 1 para `tf.compat.v1.keras.models.LinearModel` do Keras no TensorFlow 2
- Do `tf.estimator`, `Classifier` ou `Regressor` de `DNNEstimator` no TensorFlow 1 para um Keras DNN ModelKeras personalizado no TensorFlow 2
- Do `tf.estimator`, `Classifier` ou `Regressor` de `DNNLinearCombinedEstimator` no TensorFlow 1 para `tf.compat.v1.keras.models.WideDeepModel` no TensorFlow 2
- Do `tf.estimator`, `Classifier` ou `Regressor` `BoostedTreesEstimator` tf.estimator no TensorFlow 1 para `tfdf.keras.GradientBoostedTreesModel` no TensorFlow 2

Um precursor comum para o treinamento de um modelo é o pré-processamento de características, que é feito para modelos TensorFlow 1 Estimator com `tf.feature_column`. Para mais informações sobre o pré-processamento de características no TensorFlow 2, consulte [este guia sobre como migrar de colunas de características para a API de camadas de pré-processamento do Keras](migrating_feature_columns.ipynb).

## Configuração

Comece com algumas importações necessárias do TensorFlow,

In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
import keras
import pandas as pd
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import tensorflow_decision_forests as tfdf


prepare alguns dados simples para demonstração usando o dataset padrão do Titanic,

In [ ]:
x_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
x_eval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
x_train['sex'].replace(('male', 'female'), (0, 1), inplace=True)
x_eval['sex'].replace(('male', 'female'), (0, 1), inplace=True)

x_train['alone'].replace(('n', 'y'), (0, 1), inplace=True)
x_eval['alone'].replace(('n', 'y'), (0, 1), inplace=True)

x_train['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)
x_eval['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)

x_train.drop(['embark_town', 'deck'], axis=1, inplace=True)
x_eval.drop(['embark_town', 'deck'], axis=1, inplace=True)

y_train = x_train.pop('survived')
y_eval = x_eval.pop('survived')

In [ ]:
# Data setup for TensorFlow 1 with `tf.estimator`
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((dict(x_train), y_train)).batch(32)


def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices((dict(x_eval), y_eval)).batch(32)


FEATURE_NAMES = [
    'age', 'fare', 'sex', 'n_siblings_spouses', 'parch', 'class', 'alone'
]

feature_columns = []
for fn in FEATURE_NAMES:
  feat_col = tf1.feature_column.numeric_column(fn, dtype=tf.float32)
  feature_columns.append(feat_col)

e crie um método para instanciar um otimizador de amostra simplista para usar com vários modelos TensorFlow 1 Estimator e TensorFlow 2 Keras.

In [ ]:
def create_sample_optimizer(tf_version):
  if tf_version == 'tf1':
    optimizer = lambda: tf.keras.optimizers.legacy.Ftrl(
        l1_regularization_strength=0.001,
        learning_rate=tf1.train.exponential_decay(
            learning_rate=0.1,
            global_step=tf1.train.get_global_step(),
            decay_steps=10000,
            decay_rate=0.9))
  elif tf_version == 'tf2':
    optimizer = tf.keras.optimizers.legacy.Ftrl(
        l1_regularization_strength=0.001,
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=0.1, decay_steps=10000, decay_rate=0.9))
  return optimizer

## Exemplo 1: migrando do LinearEstimator

### TensorFlow 1: usando o LinearEstimator

No TensorFlow 1, você pode usar o `tf.estimator.LinearEstimator` para criar um modelo linear de referência para problemas de regressão e classificação.

In [ ]:
linear_estimator = tf.estimator.LinearEstimator(
    head=tf.estimator.BinaryClassHead(),
    feature_columns=feature_columns,
    optimizer=create_sample_optimizer('tf1'))

In [ ]:
linear_estimator.train(input_fn=_input_fn, steps=100)
linear_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

### TensorFlow 2: usando Keras LinearModel

No TensorFlow 2, você pode criar uma instância do Keras `tf.compat.v1.keras.models.LinearModel`, que é o substituto do `tf.estimator.LinearEstimator`. O caminho `tf.compat.v1.keras` é usado para indicar que o modelo pré-fabricado existe para compatibilidade.

In [ ]:
linear_model = tf.compat.v1.keras.experimental.LinearModel()
linear_model.compile(loss='mse', optimizer=create_sample_optimizer('tf2'), metrics=['accuracy'])
linear_model.fit(x_train, y_train, epochs=10)
linear_model.evaluate(x_eval, y_eval, return_dict=True)

## Exemplo 2: migrando do DNNEstimator

### TensorFlow 1: usando DNNEstimator

No TensorFlow 1, você pode usar o `tf.estimator.DNNEstimator` para criar um modelo de rede neural profunda (DNN) básico para problemas de regressão e classificação.

In [ ]:
dnn_estimator = tf.estimator.DNNEstimator(
    head=tf.estimator.BinaryClassHead(),
    feature_columns=feature_columns,
    hidden_units=[128],
    activation_fn=tf.nn.relu,
    optimizer=create_sample_optimizer('tf1'))

In [ ]:
dnn_estimator.train(input_fn=_input_fn, steps=100)
dnn_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

### TensorFlow 2: usando Keras para criar um modelo DNN personalizado

No TensorFlow 2, você pode criar um modelo DNN personalizado para substituir aquele gerado por `tf.estimator.DNNEstimator`, com níveis semelhantes de personalização especificada pelo usuário (por exemplo, como no exemplo anterior, a capacidade de personalizar um otimizador de modelo escolhido).

Um workflow semelhante pode ser usado para substituir `tf.estimator.experimental.RNNEstimator` por um modelo de rede neural recorrente Keras (RNN). O Keras fornece diversas opções integradas e personalizáveis ​​através de `tf.keras.layers.RNN`, `tf.keras.layers.LSTM` e `tf.keras.layers.GRU`. Para saber mais, veja *Camadas RNN integradas: um exemplo simples*, seção do guia [RNN com o Keras](https://www.tensorflow.org/guide/keras/rnn).

In [ ]:
dnn_model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(1)])

dnn_model.compile(loss='mse', optimizer=create_sample_optimizer('tf2'), metrics=['accuracy'])

In [ ]:
dnn_model.fit(x_train, y_train, epochs=10)
dnn_model.evaluate(x_eval, y_eval, return_dict=True)

## Exemplo 3: migrando do DNNLinearCombinedEstimator

### TensorFlow 1: usando o DNNLinearCombinedEstimator

No TensorFlow 1, você pode usar `tf.estimator.DNNLinearCombinedEstimator` para criar um modelo combinado para problemas de regressão e classificação com capacidade de personalização para seus componentes lineares e DNN.

In [ ]:
optimizer = create_sample_optimizer('tf1')

combined_estimator = tf.estimator.DNNLinearCombinedEstimator(
    head=tf.estimator.BinaryClassHead(),
    # Wide settings
    linear_feature_columns=feature_columns,
    linear_optimizer=optimizer,
    # Deep settings
    dnn_feature_columns=feature_columns,
    dnn_hidden_units=[128],
    dnn_optimizer=optimizer)

In [ ]:
combined_estimator.train(input_fn=_input_fn, steps=100)
combined_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

### TensorFlow 2: usando Keras WideDeepModel

No TensorFlow 2, você pode criar uma instância do Keras `tf.compat.v1.keras.models.WideDeepModel` para substituir uma gerada por `tf.estimator.DNNLinearCombinedEstimator`, com níveis semelhantes de personalização especificada pelo usuário (por exemplo, como no exemplo anterior, a capacidade de personalizar um otimizador de modelo escolhido).

Este `WideDeepModel` é construído com base num `LinearModel` constituinte e num modelo DNN personalizado, ambos discutidos nos dois exemplos anteriores. Um modelo linear personalizado também pode ser usado no lugar do Keras `LinearModel` integrado, se desejado.

Se você prefere construir seu próprio modelo em vez de usar um estimador pronto, confira o guia [Modelo sequencial Keras](https://www.tensorflow.org/guide/keras/sequential_model). Para obter mais informações sobre treinamento personalizado e otimizadores, veja o guia [Treinamento personalizado: passo a passo](https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough).

In [ ]:
# Create LinearModel and DNN Model as in Examples 1 and 2
optimizer = create_sample_optimizer('tf2')

linear_model = tf.compat.v1.keras.experimental.LinearModel()
linear_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
linear_model.fit(x_train, y_train, epochs=10, verbose=0)

dnn_model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(1)])
dnn_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
combined_model = tf.compat.v1.keras.experimental.WideDeepModel(linear_model,
                                                               dnn_model)
combined_model.compile(
    optimizer=[optimizer, optimizer], loss='mse', metrics=['accuracy'])
combined_model.fit([x_train, x_train], y_train, epochs=10)
combined_model.evaluate(x_eval, y_eval, return_dict=True)

## Exemplo 4: migrando do BoostedTreesEstimator

### TensorFlow 1: usando o BoostedTreesEstimator

No TensorFlow 1, você poderia usar o `tf.estimator.BoostedTreesEstimator` para criar uma base para criar um modelo de Gradient Boosting básico usando um conjunto de árvores de decisão para problemas de regressão e classificação. Essa funcionalidade não está mais incluída no TensorFlow 2.

```
bt_estimator = tf1.estimator.BoostedTreesEstimator(
    head=tf.estimator.BinaryClassHead(),
    n_batches_per_layer=1,
    max_depth=10,
    n_trees=1000,
    feature_columns=feature_columns)
```

```
bt_estimator.train(input_fn=_input_fn, steps=1000)
bt_estimator.evaluate(input_fn=_eval_input_fn, steps=100)
```

### TensorFlow 2: usando Florestas de Decisão do TensorFlow

No TensorFlow 2, o `tf.estimator.BoostedTreesEstimator` é substituído por [tfdf.keras.GradientBoostedTreesModel](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/GradientBoostedTreesModel#attributes) do pacote [TensorFlow Decision Forests](https://www.tensorflow.org/decision_forests).

As Florestas de Decisão do TensorFlow oferecem diversas vantagens sobre o `tf.estimator.BoostedTreesEstimator`, principalmente em relação à qualidade, velocidade, facilidade de uso e flexibilidade. Para saber mais sobre as florestas de decisão do TensorFlow, comece com o [beginner colab](https://www.tensorflow.org/decision_forests/tutorials/beginner_colab).

O exemplo a seguir mostra como treinar um modelo Gradient Boosted Trees usando o TensorFlow 2:

Instale as Florestas de Decisão do TensorFlow.

In [ ]:
!pip install tensorflow_decision_forests

Crie um dataset do TensorFlow. Observe que as Florestas de Decisão oferecem suporte nativo a muitos tipos de recursos e não precisam de pré-processamento.

In [ ]:
train_dataframe = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
eval_dataframe = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

# Convert the Pandas Dataframes into TensorFlow datasets.
train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train_dataframe, label="survived")
eval_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(eval_dataframe, label="survived")

Treine o modelo no dataset `train_dataset`.

In [ ]:
# Use the default hyper-parameters of the model.
gbt_model = tfdf.keras.GradientBoostedTreesModel()
gbt_model.fit(train_dataset)

Avalie a qualidade do modelo no dataset `eval_dataset`.

In [ ]:
gbt_model.compile(metrics=['accuracy'])
gbt_evaluation = gbt_model.evaluate(eval_dataset, return_dict=True)
print(gbt_evaluation)

O Gradient Boosted Trees é apenas um dos muitos algoritmos de floresta de decisão disponíveis no TensorFlow Decision Forests. Por exemplo, Random Forests (florestas aleatórias), disponível como [tfdf.keras.GradientBoostedTreesModel](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/RandomForestModel) é muito resistente ao overfitting, enquanto CART (disponível como [tfdf.keras.CartModel](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/CartModel)) é ótimo para a interpretação de modelos.

No próximo exemplo, treine e plote um modelo Random Forest.

In [ ]:
# Train a Random Forest model
rf_model = tfdf.keras.RandomForestModel()
rf_model.fit(train_dataset)

# Evaluate the Random Forest model
rf_model.compile(metrics=['accuracy'])
rf_evaluation = rf_model.evaluate(eval_dataset, return_dict=True)
print(rf_evaluation)

No exemplo final, treine e avalie um modelo CART.

In [ ]:
# Train a CART model
cart_model = tfdf.keras.CartModel()
cart_model.fit(train_dataset)

# Plot the CART model
tfdf.model_plotter.plot_model_in_colab(cart_model, max_depth=2)